# AutoEncoders


get the dataset

In [ ]:
movies = pd.read_csv('datasets/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
# MovieID::Title::Genres
movies.head()

In [ ]:
users = pd.read_csv('datasets/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
# UserID::Gender::Age::Occupation::Zip-code
users.head()

In [ ]:
ratings = pd.read_csv('datasets/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
# UserID::MovieID::Rating::Timestamp
ratings.head()

In [ ]:
train-test set prep

In [ ]:
X_train = pd.read_csv('datasets/ml-100k/u1.base', delimiter='\t')
X_test  = pd.read_csv('datasets/ml-100k/u1.test', delimiter='\t')

X_train = np.array(X_train, dtype='int')
X_test  = np.array(X_test, dtype='int')

# UserID::MovieID::Rating::Timestamp
X_test[0:5]

In [ ]:
user_count  = int(max(max(X_train[:,0]), max(X_test[:,0])))
movie_count = int(max(max(X_train[:,1]), max(X_test[:,1])))
print("user_count:  ", user_count)
print("movie_count: ", movie_count)